In [1]:
import glob
import pandas as pd
import urllib
import requests
import xml.etree.ElementTree as ET
import time
import numpy as np
from itertools import zip_longest

In [2]:
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [3]:
df = pd.read_csv('/home/riddleta/ac_knowl/output/file_index.csv')

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,file,pmcid,paper_number
0,/home/riddleta/ac_knowl/output/full_texts/pape...,3972225,0
1,/home/riddleta/ac_knowl/output/full_texts/pape...,3972226,1
2,/home/riddleta/ac_knowl/output/full_texts/pape...,3972227,2
3,/home/riddleta/ac_knowl/output/full_texts/pape...,3972228,3
4,/home/riddleta/ac_knowl/output/full_texts/pape...,3972229,4


In [5]:
with open('api_key.txt', 'r') as infile:
    line = infile.readlines()
    api_key = line[0][10:]

In [ ]:
mesh_labs = []
mesh_ids = []
grant_ids = []
grant_agency = []
journal_out = []
pmid_out = []
auths_out = []
pmcid_out = []
total_papes = 0
for i in grouper(df.pmcid.tolist(), 200):
    try:
        i = list(map(str, i)) #ensure strings
        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&id=' + str(','.join(i)) + '&retmode=json&api_key=' + str(api_key)
        r = requests.get(pth)
        r_out = r.json()
        jr=[] #store local group results
        pmr=[]
        ar=[]
        pmcr=[]
        for j in r_out['result']['uids']:
            jr.append(r_out['result'][str(j)]['fulljournalname'])
            pmr.append([art_id['value'] for art_id in r_out['result'][str(j)]['articleids'] if art_id['idtype']=='pmid'][0])
            ar.append(r_out['result'][str(j)]['authors'])
            pmcr.append(j)


        journal_out.extend(jr) #build up lists for writing
        pmid_out.extend(pmr)
        auths_out.extend(ar)
        pmcid_out.extend(pmcr)

        out_df = pd.DataFrame({'journal':journal_out,
                          'pmid':pmid_out,
                          'auths':auths_out,
                          'pmcid':pmcid_out})
        if out_df.shape[0]>100000:
            total_papes = total_papes + out_df.shape[0]
            fn = 'grant_id_'
            out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(total_papes)+'.csv', index=False)
            journal_out = []
            pmid_out = []
            auths_out = []
            pmcid_out = []

        time.sleep(.11)
    except Exception as e:
        print(str(i[0])+'*******'+str(i[199]))
        print(e)
        time.sleep(.11)


5846053*******5846376
'fulljournalname'
6450715*******6456894
'fulljournalname'
5390906*******5391162
'fulljournalname'
4775292*******4776336
'fulljournalname'
5929106*******5931393
'fulljournalname'
5822532*******5822787
'fulljournalname'
5819009*******4370912
Expecting value: line 1 column 1 (char 0)
4370913*******4371426
Expecting value: line 1 column 1 (char 0)
4107366*******4108450
'fulljournalname'
4015299*******4015605
'fulljournalname'
5131257*******5133195
'fulljournalname'
4156544*******4157118
'fulljournalname'
5727783*******5728061
'fulljournalname'


In [ ]:
total_papes

In [27]:
list(map(str, i))

['6057084',
 '4901318',
 '5066403',
 '4963138',
 '4921636',
 '5021966',
 '5009682',
 '5146782',
 '5047131',
 '6060497',
 '4861734',
 '6058380',
 '4927541',
 '6059306',
 '5154497',
 '4866476',
 '4893845',
 '5168700',
 '4956923',
 '4930344',
 '4906879',
 '4867884',
 '5187529',
 '5008858',
 '5036301',
 '4880642',
 '5023661',
 '5057336',
 '4919838',
 '5010214',
 '6034155',
 '4977644',
 '5168440',
 '4929888',
 '4902956',
 '5041572',
 '5105114',
 '5147497',
 '5053639',
 '5135046',
 '5148605',
 '4881716',
 '6036431',
 '4918678',
 '4935726',
 '4994429',
 '4969043',
 '4879346',
 '4913953',
 '6038603',
 '5001952',
 '6054685',
 '4929459',
 '4909765',
 '4938287',
 '4987053',
 '4940342',
 '5054885',
 '5020311',
 '5014317',
 '5115840',
 '5037525',
 '4937531',
 '6051175',
 '5011657',
 '5065158',
 '5093585',
 '4867921',
 '4919548',
 '5190020',
 '5067575',
 '3166172',
 '5112819',
 '4981418',
 '5029226',
 '5164879',
 '6030377',
 '5083719',
 '4950111',
 '4873622',
 '5007702',
 '5168825',
 '5059142',
 '48

In [14]:
#pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&id=' + str(','.join(i)) + '&retmode=json&api_key=' + str(api_key)
#r = requests.get(pth)
#r_out = r.json()
r = requests.get(pth)

In [23]:
type(i[1])

int

### everything below here is olllld
next time, i gotta do the grant api

In [4]:
status_update = [i for i in range(50000, df.shape[0], 50000)]

[50000,
 100000,
 150000,
 200000,
 250000,
 300000,
 350000,
 400000,
 450000,
 500000,
 550000,
 600000,
 650000,
 700000,
 750000,
 800000,
 850000,
 900000,
 950000,
 1000000,
 1050000,
 1100000,
 1150000,
 1200000,
 1250000,
 1300000,
 1350000,
 1400000,
 1450000,
 1500000,
 1550000,
 1600000,
 1650000,
 1700000,
 1750000,
 1800000,
 1850000,
 1900000,
 1950000,
 2000000,
 2050000,
 2100000,
 2150000,
 2200000,
 2250000,
 2300000,
 2350000,
 2400000,
 2450000,
 2500000,
 2550000]

In [ ]:
mesh_labs = []
mesh_ids = []
grant_ids = []
grant_agency = []
journal_out = []
pmid_out = []
auths_out = []
pmcid_out = []
for j, pmcid in enumerate(df.pmcid.tolist()):
    pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pmc&id=' + str(pmcid) + '&retmode=json&api_key=' + str(api_key)
    try:
        r = requests.get(pth)
        r_out = r.json()
        journal = r_out['result'][str(pmcid)]['fulljournalname']
        pmid = [art_id['value'] for art_id in r_out['result'][str(pmcid)]['articleids'] if art_id['idtype']=='pmid'][0]
        auths = r_out['result'][str(pmcid)]['authors']
        time.sleep(.11)

        pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=' + pmid + '&retmode=xml&api_key=' + str(api_key)
        r2 = requests.get(pth)
        root = ET.fromstring(r2.content)
        tmp_mesh_labs = []
        tmp_mesh_ids = []
        tmp_grant_ids = []
        tmp_grant_agency = []
        for mesh in root.iter('MeshHeading'):
            tmp_mesh_labs.append([i.text for i in mesh])
            tmp_mesh_ids.append([i.attrib['UI'] for i in mesh])
        for grant in root.iter('Grant'):
            tmp_grant_ids.append([i.text for i in grant if i.tag=='GrantID'])
            tmp_grant_agency.append([i.text for i in grant if i.tag=='Agency'])

        journal_out.append(journal)
        pmid_out.append(pmid)
        auths_out.append(auths)
        mesh_labs.append([lab for entry in tmp_mesh_labs for lab in entry])
        mesh_ids.append([lab for entry in tmp_mesh_ids for lab in entry])
        grant_ids.append(tmp_grant_ids)
        grant_agency.append(tmp_grant_agency)
        pmcid_out.append(pmcid)
        time.sleep(.1)
    except:
        journal_out.append(np.nan)
        pmid_out.append(np.nan)
        auths_out.append(np.nan)
        mesh_labs.append(np.nan)
        mesh_ids.append(np.nan)
        grant_ids.append(np.nan)
        grant_agency.append(np.nan)
        pmcid_out.append(np.nan)
    
    if j in status_update:
        out_df = pd.DataFrame({'journal':journal_out,
                               'pmid':pmid_out,
                               'auths':auths_out,
                               'mesh_labs':mesh_labs,
                               'mesh_ids':mesh_ids,
                               'grant_ids':grant_ids,
                               'grant_agency':grant_agency,
                               'pmcid':pmcid_out})
        fn = 'grant_id_'
        out_df.to_csv('/home/riddleta/ac_knowl/output/grant_identifiers/'+fn+str(j)+'.csv', index=False)
        mesh_labs = []
        mesh_ids = []
        grant_ids = []
        grant_agency = []
        journal_out = []
        pmid_out = []
        auths_out = []
        pmcid_out = []
        
        

In [200]:
root.findall('.//Grant')

[]

In [105]:
pmid = 24691291
pth = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=' + str(pmid) + '&retmode=xml'
r2 = requests.get(pth)
root = ET.fromstring(r2.content)

NameError: name 'grant_ids' is not defined

In [160]:
grant[0][0]

IndexError: child index out of range

In [135]:
for child in grant.getchildren():
    print(child.attrib['GrantID'])

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """Entry point for launching an IPython kernel.


KeyError: 'GrantID'

In [102]:
for mesh in root.iter('MeshHeading'):
    print([i.attrib['MajorTopicYN'] for i in mesh])
    print([i.text for i in mesh])
    print([i.attrib['UI'] for i in mesh])

['N']
['Adolescent']
['D000293']
['N']
['Adult']
['D000328']
['N', 'Y']
['Asian Continental Ancestry Group', 'genetics']
['D044466', 'Q000235']
['N']
['China']
['D002681']
['N', 'N']
['Ethnic Groups', 'genetics']
['D005006', 'Q000235']
['N']
['Female']
['D005260']
['N', 'Y']
['Gene Frequency', 'genetics']
['D005787', 'Q000235']
['N']
['Geography']
['D005843']
['N', 'Y']
['HLA-A Antigens', 'genetics']
['D015234', 'Q000235']
['N', 'Y']
['HLA-B Antigens', 'genetics']
['D015235', 'Q000235']
['N', 'Y']
['HLA-DRB1 Chains', 'genetics']
['D059811', 'Q000235']
['N', 'Y']
['Haplotypes', 'genetics']
['D006239', 'Q000235']
['N']
['Humans']
['D006801']
['N']
['Linkage Disequilibrium']
['D015810']
['N']
['Male']
['D008297']
['N']
['Middle Aged']
['D008875']
['N']
['Minority Groups']
['D008913']
['N']
['Phylogeny']
['D010802']
['N']
['Tissue Donors']
['D014019']
['N']
['Young Adult']
['D055815']
